In [1]:
NEWS_API_KEY = "c07df8265f264b20a0c2646a1740193e"
import requests
import json

import pandas as pd
from pandas.io.json import json_normalize
pd.options.display.max_rows = 10
import re

In [69]:
def get_all_articles(topic, date):
    page_number = 1
    all_articles = pd.DataFrame({})
    while True:
        url = ('https://newsapi.org/v2/everything?'
           'q=%s&'
           'from=%s&'
           'page=%d&'
           'language=en&'  # language is english
           'pageSize=100&'  # maximum page size
           'sortBy=popularity&'
           'apiKey=%s'
        ) % (topic, date, page_number, NEWS_API_KEY)
        response = requests.get(url)
        news = json.loads(response.text)
        if not news["articles"]:
            break
        all_articles = pd.concat([all_articles, json_normalize(news["articles"])], ignore_index=True)
        page_number += 1
        print(len(all_articles))
    return all_articles

def clean_articles(df):
    df["Date"] = df.publishedAt.str.split("T").str[0]
    # Do more text cleaning here
    df["description"].dropna()
    df["label"] = None
    df["description"] = df.description.str.replace('[^a-z A-Z]', '').str.lower().str.split()
    return df

In [98]:
articles = get_all_articles("apple", "2018-06-11")

100
200
300
400
500
600
700
800
898


In [99]:
clean_articles(articles)

,author,description,publishedAt,source.id,source.name,title,url,urlToImage,Date,label
0,Adi Robertson,"[the, fcc, is, repealing, net, neutrality, rul...",2018-06-11T16:18:20Z,the-verge,The Verge,Here’s how companies have flouted net neutrali...,https://www.theverge.com/2018/6/11/17438638/ne...,https://cdn.vox-cdn.com/thumbor/qI4zFeJu5KvJ4b...,2018-06-11,None
1,Sarah Perez,"[facebook, may, have, disbanded, its, trending...",2018-06-11T18:03:01Z,techcrunch,TechCrunch,Facebook still wants to be a media company,http://techcrunch.com/2018/06/11/facebook-stil...,https://techcrunch.com/wp-content/uploads/2017...,2018-06-11,None
2,Ingrid Lunden,"[memrise, a, uk, startup, whose, eponymous, la...",2018-06-11T09:31:46Z,techcrunch,TechCrunch,Memrise raises $15.5M as its AI-based language...,http://techcrunch.com/2018/06/11/memrise-raise...,https://techcrunch.com/wp-content/uploads/2014...,2018-06-11,None
3,Anthony Ha,"[prhbtd, a, startup, that, ceo, drake, suttons...",2018-06-11T17:44:46Z,techcrunch,TechCrunch,Media startup Prøhbtd raises $8M to help bring...,http://techcrunch.com/2018/06/11/prohbtd-serie...,https://techcrunch.com/wp-content/uploads/2018...,2018-06-11,None
4,Sam Rutherford,"[with, the, price, of, bitcoin, approaching, a...",2018-06-11T18:45:00Z,None,Gizmodo.com,Apple's New Guidelines for the iOS App Store I...,https://gizmodo.com/apples-new-guidelines-for-...,https://i.kinja-img.com/gawker-media/image/upl...,2018-06-11,None
...,...,...,...,...,...,...,...,...,...,...
893,Dendiweik,"[joost, is, a, new, way, of, watching, tv, on,...",2018-06-11T06:27:10Z,None,Pickthebrain.com,Comment on Joost Beta Preview: The Best TV Onl...,https://www.pickthebrain.com/blog/joost-beta-p...,https://www.pickthebrain.com/blog/wp-content/i...,2018-06-11,None
894,tiara tech,"[thanks, for, sharing, valuable, information, ...",2018-06-11T10:44:00Z,None,Coenraets.org,Comment on Sample Force.com Mobile Application...,http://coenraets.org/blog/2014/02/sample-force...,None,2018-06-11,None
895,Danitra,"[ive, found, that, coconut, oil, is, my, skins...",2018-06-11T17:40:41Z,None,Helloglow.co,Comment on 2-Ingredient DIY Toner Recipes for ...,https://helloglow.co/2-ingredient-diy-toner-re...,https://helloglow.co/wp-content/uploads/2014/0...,2018-06-11,None
896,Harry Guinness,"[there, are, dozens, of, factors, that, can, a...",2018-06-11T12:00:00Z,None,Howtogeek.com,What to Do If Your iPhone Is Getting a Bad Signal,https://www.howtogeek.com/355014/what-to-do-if...,https://www.howtogeek.com/thumbcache/2/200/d8c...,2018-06-11,None


In [100]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

In [101]:
training = articles.sample(frac=0.5)["description"]
testing = articles[~articles.index.isin(training.index)]["description"]
testing

1      [facebook, may, have, disbanded, its, trending...
4      [with, the, price, of, bitcoin, approaching, a...
6      [mobile, world, congress, is, one, of, those, ...
12     [frances, president, emmanuel, macron, is, woo...
13     [the, insider, picks, team, writes, about, stu...
                             ...                        
891    [child, advocacy, group, urges, ftc, to, probe...
892    [common, sense, media, is, filing, a, new, com...
893    [joost, is, a, new, way, of, watching, tv, on,...
895    [ive, found, that, coconut, oil, is, my, skins...
897    [absolutely, fantastic, postingquickbooks, pos...
Name: description, Length: 449, dtype: object

In [127]:
# Each document is represented by a tuple (sentence, label). 
# The sentence is tokenized, so it is represented by a list of strings

training_docs = [(x, 'subj') for x in training.values if x]
testing_docs = [(x, 'subj') for x in testing.values  if x]
training_docs[0]

(['pride',
  'month',
  'is',
  'here',
  'and',
  'although',
  'equality',
  'and',
  'diversity',
  'should',
  'be',
  'celebrated',
  'every',
  'day',
  'june',
  'highlights',
  'its',
  'importance',
  'aside',
  'from',
  'speaking',
  'up',
  'about',
  'issues',
  'and',
  'taking',
  'a',
  'stand',
  'you',
  'can',
  'also',
  'show',
  'your',
  'support',
  'by',
  'copping',
  'merchandise',
  'as',
  'part',
  'of',
  'brands',
  'pride',
  'coll'],
 'subj')

In [106]:
import itertools
# all_words = list(itertools.chain.from_iterable([row for row in articles.description if row]))
# training_words = list(itertools.chain.from_iterable([row for row in training if row]))
# testing_words = list(itertools.chain.from_iterable([row for row in testing if row]))

In [129]:
from nltk.corpus import stopwords
  
sentim_analyzer = SentimentAnalyzer()
stopwords = stopwords.words('english')

all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)

In [130]:
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)

In [131]:
training_set = sentim_analyzer.apply_features(training_docs)
test_set = sentim_analyzer.apply_features(testing_docs)

trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)

Training classifier


In [132]:
sentim_analyzer.evaluate(test_set)

Evaluating NaiveBayesClassifier results...


{'Accuracy': 1.0,
 'Precision [subj]': 1.0,
 'Recall [subj]': 1.0,
 'F-measure [subj]': 1.0}

In [133]:
sentim_analyzer.evaluate(test_set)

Evaluating NaiveBayesClassifier results...


{'Accuracy': 1.0,
 'Precision [subj]': 1.0,
 'Recall [subj]': 1.0,
 'F-measure [subj]': 1.0}